In [1]:
#Importando todas as nossas libraries:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils as np_utils
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

In [2]:
#Vamos limpar nossas bases de dados:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [3]:
#Atribuindo valores mais comuns para valores nulos como "Na" na variavel valores:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell', 
           'model': 'golf', 'fuelType': 'benzin', 
           'notRepairedDamage': 'nein'}

In [4]:
#Preenchendo todos os campos com "Na" dentro da nossa base de dados:
base = base.fillna(value = valores)

In [5]:
#Atribuindo valores as nossas variáveis: Previsores e Preço Real
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [6]:
#Transformando todos os nossos dados listados em numeros através do Label Encorder:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 10] = labelencoder_previsores.fit_transform(previsores[:, 10])

In [7]:
#Transformando todos os nossos dados listados em dados entre 0 e 1 através do One Hot Encorder:
categorical_columns = [0, 1, 3, 5, 8, 9, 10]
onehotencoder = ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorical_columns)],
    remainder='passthrough')
data_transformed = ct.fit_transform(previsores).toarray()

In [8]:
#Criando nossa rede neural:
def criarRede():
    k.clear_session()
    regressor = Sequential([
    tf.keras.layers.InputLayer(shape=(11,)),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])
    return regressor

In [9]:
#Regressor recebe 
regressor = KerasRegressor(model= criarRede, epochs = 100, batch_size = 300)

In [10]:
resultados = cross_val_score(estimator=regressor, X=previsores, y=preco_real, cv=10, scoring='neg_mean_absolute_error')


Epoch 1/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 27265.7734 - mean_absolute_error: 27265.7734
Epoch 2/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 16983.4727 - mean_absolute_error: 16983.4727
Epoch 3/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 15756.3066 - mean_absolute_error: 15756.3066
Epoch 4/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 16844.5625 - mean_absolute_error: 16844.5625
Epoch 5/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 41576.6992 - mean_absolute_error: 41576.6992
Epoch 6/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 19082.5410 - mean_absolute_error: 19082.5410
Epoch 7/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 8482.6523 - mean_absolute_error: 8482.6523
Epoch 8/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 12250.5713 - mean_absolute_error: 12250.5713
Epoch 9/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 10827.0674 - mean_absolute_error: 10827.0674
Epoch 10/100
1115/1115 ━━━━━━

KeyboardInterrupt: 